In [94]:
import numpy as np
from qecsim import paulitools as pt
from qecsim.models.generic import BitFlipErrorModel
from qecsim.models.toric import ToricCode, ToricMWPMDecoder
import math
import itertools
import more_itertools as mit

# set size of lattice
L = 19
num_qubits = 2*(L**2)
num_sigma = 2

# initialise models
my_code = ToricCode(L,L)
my_error_model = BitFlipErrorModel()
my_decoder = ToricMWPMDecoder()

# set physical error probability to 1%
error_probability = 0.001

# init variables
error_prob = dict()
syndrome_prob = dict()
succ_prob = dict()
succ_prob["success"] = 0
succ_prob["fail"] = 0
syndrome_succ_prob = dict()

# seed random number generator for repeatability (leave blank for random)
rng = np.random.default_rng()

# we approximate the binomial distribution as gaussian and calculate the maximum number of errors up to mu + 3 sigma
bits = [0,1]
mu = num_qubits * error_probability
sigma = np.sqrt(num_qubits * error_probability * (1-error_probability))
max_errors = int(np.floor(mu + num_sigma * sigma))

base_error_vec = []
for i in range(0,max_errors):
    tmp = np.zeros(num_qubits,dtype=int)
    np.put(tmp,range(0,i+1),np.ones(i+1))
    base_error_vec.append(tmp)

errors = []
for i in base_error_vec:
    errors = errors + list(mit.distinct_permutations(i,num_qubits))

In [95]:
for values in errors:
    # chuck zeros on the end of the error vector since error is just bit flip (so we ignore phase flips)
    arr = np.zeros(2*(L**2),dtype=int)
    values = np.array(values)
    error = np.concatenate((values, arr))
    #error = my_error_model.generate(my_code, error_probability, rng)
    prob = error_probability**(sum(values)) * (1-error_probability)**(num_qubits-sum(values))

    # syndrome: stabilizers that do not commute with the error
    syndrome = pt.bsp(error, my_code.stabilizers.T)

    recovery = my_decoder.decode(my_code, syndrome)
    is_correct = sum(pt.bsp(recovery ^ error, my_code.logicals.T))

    error = str(error)
    syndrome = str(syndrome)

    # update error prob
    error_prob[error] = prob

    # update syndrome prob
    if syndrome in syndrome_prob:
        syndrome_prob[syndrome] = syndrome_prob[syndrome] + prob
    else:
        syndrome_prob[syndrome] = prob

    # update EC prob
    if is_correct == 0:
        succ_prob["success"] = succ_prob["success"] + prob
        syndrome_succ = syndrome + "1"
    else:
        succ_prob["fail"] = succ_prob["fail"] + prob
        syndrome_succ = syndrome + "0"

    # update joint S, EC prob
    if syndrome_succ in syndrome_succ_prob:
        syndrome_succ_prob[syndrome_succ] = syndrome_succ_prob[syndrome_succ] + prob
    else:
        syndrome_succ_prob[syndrome_succ] = prob
else:
    He = 0
    for error in error_prob:
        p = error_prob[error]
        He = He - p*math.log2(p)

    Hs = 0
    for syndrome in syndrome_prob:
        p = syndrome_prob[syndrome]
        Hs = Hs - p*math.log2(p)
    
    Hec = 0
    for succ in succ_prob:
        p = succ_prob[succ]
        if p != 0:
            Hec = Hec - p*math.log2(p)

    Hsec = 0
    for syndrome_succ in syndrome_succ_prob:
        p = syndrome_succ_prob[syndrome_succ]
        Hsec = Hsec - p*math.log2(p)

    print("L = " + str(L))
    print("H(E) = " + str(He))
    print("H(S) = " + str(Hs))
    print("H(EC) = " + str(Hec))
    print("H(S,EC) = " + str(Hsec))
    print("I(S;EC) = " + str(Hs + Hec - Hsec))

L = 19
H(E) = 7.142745163301253e-05
H(S) = 6.517639627647001
H(EC) = 0.5091799266690905
H(S,EC) = 6.517639627647001
I(S;EC) = 0.5091799266690904
